In [12]:
import mediapipe as mp
import pandas as pd
import numpy as np
import cv2


In [13]:
mp_pose = mp.solutions.pose


In [14]:
landmarks_points = {
    "nose": 0,
    "left_shoulder": 11, "right_shoulder": 12,
    "left_elbow": 13, "right_elbow": 14,
    "left_wrist": 15, "right_wrist": 16,
    "left_hip": 23, "right_hip": 24,
    "left_knee": 25, "right_knee": 26,
    "left_ankle": 27, "right_ankle": 28,
    "left_heel": 29, "right_heel": 30,
    "left_foot_index": 31, "right_foot_index": 32,
}

col_names = []
for i in range(len(landmarks_points.keys())):
    name = list(landmarks_points.keys())[i]
    col_names.append(name + "_x")
    col_names.append(name + "_y")
    col_names.append(name + "_z")
    col_names.append(name + "_v")

pose_name = col_names.copy()

pose_name.append("pose")


In [15]:
def extract_landmarks(image):
    pre_list = []
    with mp_pose.Pose(static_image_mode=True, enable_segmentation=True) as pose:
        result = pose.process(
            cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        try:
            for landmarks in result.pose_landmarks.landmark:
                pre_list.append(landmarks)
            predict = True
        except AttributeError:
            print(f"No points")
            predict = False

    if predict == True:
        gen1116 = np.array([
            [
                pre_list[m].x,
                pre_list[m].y,
                pre_list[m].z,
                pre_list[m].visibility
            ] for m in range(11, 17)
        ]).flatten().tolist()

        gen2333 = np.array([
            [
                pre_list[m].x,
                pre_list[m].y,
                pre_list[m].z,
                pre_list[m].visibility
            ] for m in range(23, 33)
        ]).flatten().tolist()

        gen1116.extend(gen2333)

        all_list = [
            pre_list[0].x,
            pre_list[0].y,
            pre_list[0].z,
            pre_list[0].visibility,
        ]

        all_list.extend(gen1116)
        data_pose = pd.DataFrame(all_list, columns=pose_name)


In [16]:
import pickle as pk
features = data_pose.drop(["pose"], axis=1)
target = data_pose[["pose"]]


loaded_model = pk.load(open("./models/11_poses.model", "rb"))
result = loaded_model.predict(features)
print(result)


[7]


In [17]:
landmarks_points = {
    "left_shoulder": [], "right_shoulder": [],
    "left_elbow": [], "right_elbow": [],
    "left_wrist": [], "right_wrist": [],
    "left_hip": [], "right_hip": [],
    "left_knee": [], "right_knee": [],
    "left_ankle": [], "right_ankle": [],
    "left_heel": [], "right_heel": [],
    "left_foot_index": [], "right_foot_index": [],
}


In [18]:
def angle(p1, p2, p3):
    a = np.array([p1[0], p1[1]])
    b = np.array([p2[0], p2[1]])
    c = np.array([p3[0], p3[1]])

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - \
        np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180:
        angle = 360 - angle

    return angle


In [19]:
for i in range(0, 11):
    for index, row in data_pose.iterrows():
        landmarks_points["left_shoulder"] = [
            row["left_shoulder_x"], row["left_shoulder_y"]]

        landmarks_points["right_shoulder"] = [
            row["right_shoulder_x"], row["right_shoulder_y"]]

        landmarks_points["left_elbow"] = [
            row["left_elbow_x"], row["left_elbow_y"]]

        landmarks_points["right_elbow"] = [
            row["right_elbow_x"], row["right_elbow_y"]]

        landmarks_points["left_wrist"] = [
            row["left_wrist_x"], row["left_wrist_y"]]

        landmarks_points["right_wrist"] = [
            row["right_wrist_x"], row["right_wrist_y"]]

        landmarks_points["left_hip"] = [
            row["left_hip_x"], row["left_hip_y"]]

        landmarks_points["right_hip"] = [
            row["right_hip_x"], row["right_hip_y"]]

        landmarks_points["left_knee"] = [
            row["left_knee_x"], row["left_knee_y"]]

        landmarks_points["right_knee"] = [
            row["right_knee_x"], row["right_knee_y"]]

        landmarks_points["left_ankle"] = [
            row["left_ankle_x"], row["left_ankle_y"]]

        landmarks_points["right_ankle"] = [
            row["right_ankle_x"], row["right_ankle_y"]]

        landmarks_points["left_heel"] = [
            row["left_heel_x"], row["left_heel_y"]]

        landmarks_points["right_heel"] = [
            row["right_heel_x"], row["right_heel_y"]]

        landmarks_points["left_foot_index"] = [
            row["left_foot_index_x"], row["left_foot_index_y"]]

        landmarks_points["right_foot_index"] = [
            row["right_foot_index_x"], row["right_foot_index_y"]]

        armpit_left = angle(
            landmarks_points["left_elbow"],
            landmarks_points["left_shoulder"],
            landmarks_points["left_hip"]
        )
        armpit_right = angle(
            landmarks_points["right_elbow"],
            landmarks_points["right_shoulder"],
            landmarks_points["right_hip"]
        )

        elbow_left = angle(
            landmarks_points["left_shoulder"],
            landmarks_points["left_elbow"],
            landmarks_points["left_wrist"]
        )
        elbow_right = angle(
            landmarks_points["right_shoulder"],
            landmarks_points["right_elbow"],
            landmarks_points["right_wrist"]
        )

        hip_left = angle(
            landmarks_points["right_hip"],
            landmarks_points["left_hip"],
            landmarks_points["left_knee"]
        )
        hip_right = angle(
            landmarks_points["left_hip"],
            landmarks_points["right_hip"],
            landmarks_points["right_knee"]
        )

        knee_left = angle(
            landmarks_points["left_hip"],
            landmarks_points["left_knee"],
            landmarks_points["left_ankle"]
        )
        knee_right = angle(
            landmarks_points["right_hip"],
            landmarks_points["right_knee"],
            landmarks_points["right_ankle"]
        )

        ankle_left = angle(
            landmarks_points["left_knee"],
            landmarks_points["left_ankle"],
            landmarks_points["left_foot_index"]
        )
        ankle_right = angle(
            landmarks_points["right_knee"],
            landmarks_points["right_ankle"],
            landmarks_points["right_foot_index"]
        )

        tmp = [
            armpit_left,
            armpit_right,
            elbow_left,
            elbow_right,
            hip_left,
            hip_right,
            knee_left,
            knee_right,
            ankle_left,
            ankle_right
        ]


In [20]:
print(tmp)


[163.11088331275798, 155.66448486441854, 105.3328965862917, 110.01028363014284, 106.09841671167501, 108.84290898941943, 176.29892937489714, 84.63853477260875, 171.8726195119761, 140.2247397307341]
